In [40]:
import subprocess
import pandas as pd
import sqlite3
import numpy as np

In [2]:
path_sqlite3 = '/Users/emily/anaconda3/bin/sqlite3'

path_DisGeNET_sqlite_file = '/Users/emily/Desktop/DisGeNET/DisGeNET_source_data/sqlite/disgenet_2020.db'
path_DisGeNET_sqlite_output_file = '/Users/emily/Desktop/DisGeNET/DisGeNET_source_data/sqlite/tables/table_names.csv'

In [3]:
connection_to_sqlite3 = sqlite3.connect(path_DisGeNET_sqlite_file)

In [4]:
sql_tables = "SELECT name FROM sqlite_master WHERE type='table';"
df_table_names = pd.read_sql_query(sql_tables, connection_to_sqlite3)

In [5]:
df_table_names

,name
0,diseaseAttributes
1,diseaseClass
2,disease2class
3,geneAttributes
4,geneDiseaseNetwork
5,variantAttributes
6,variantGene
7,variantDiseaseNetwork


In [6]:
dict_all_tables = {}
for table in df_table_names['name']:
    dict_all_tables[table] = pd.read_sql_query('SELECT * from ' + table, connection_to_sqlite3)

In [7]:
connection_to_sqlite3.close()

In [8]:
dict_all_tables.keys()

dict_keys(['diseaseAttributes', 'diseaseClass', 'disease2class', 'geneAttributes', 'geneDiseaseNetwork', 'variantAttributes', 'variantGene', 'variantDiseaseNetwork'])

## diseaseAttributes table

In [9]:
dict_all_tables['diseaseAttributes']

,diseaseNID,diseaseId,diseaseName,type
0,1,C0000727,"Abdomen, Acute",phenotype
1,2,C0000729,Abdominal Cramps,phenotype
2,3,C0000731,Abdomen distended,phenotype
3,4,C0000734,Abdominal mass,phenotype
4,5,C0000735,Abdominal Neoplasms,group
...,...,...,...,...
30288,30289,C4755314,Autosomal recessive cutis laxa type 2B,disease
30289,30290,C4757950,Isolated ATP synthase deficiency,disease
30290,30291,C4757951,Desmoplastic infantile astrocytoma and ganglio...,disease
30291,30292,C4759295,Non-metastatic prostate cancer,disease


In [10]:
dict_all_tables['diseaseAttributes']['type'].unique()

array(['phenotype', 'group', 'disease'], dtype=object)

## diseaseClass table

In [11]:
dict_all_tables['diseaseClass']

,diseaseClassNID,vocabulary,diseaseClass,diseaseClassName
0,2,MSH,C23,"Pathological Conditions, Signs and Symptoms"
1,3,MSH,C16,"Congenital, Hereditary, and Neonatal Diseas..."
2,6,MSH,C06,Digestive System Diseases
3,7,MSH,C04,Neoplasms
4,10,MSH,C18,Nutritional and Metabolic Diseases
5,17,MSH,C13,Female Urogenital Diseases and Pregnancy Co...
6,20,MSH,C01,Infections
7,21,MSH,C11,Eye Diseases
8,22,MSH,C17,Skin and Connective Tissue Diseases
9,25,MSH,C05,Musculoskeletal Diseases


In [13]:
dict_all_tables['diseaseClass']['vocabulary'].unique()

array(['MSH'], dtype=object)

## disease2class (disease-to-class) table 

In [17]:
dict_all_tables['disease2class']

,diseaseNID,diseaseClassNID
0,1840,20
1,38,20
2,8033,20
3,13566,20
4,39,20
...,...,...
38319,20669,37
38320,40,37
38321,5209,37
38322,4771,37


#### Testing an idea:

In [30]:
df_emily_disease_to_class = (
    pd.merge(
        dict_all_tables['disease2class'],
        dict_all_tables['diseaseClass'],
        on = ['diseaseClassNID'],
        how = 'left',
    )
    .rename(columns = {'vocabulary' : 'diseaseClass_vocabulary'})
)

In [31]:
df_emily_disease_to_class

,diseaseNID,diseaseClassNID,diseaseClass_vocabulary,diseaseClass,diseaseClassName
0,1840,20,MSH,C01,Infections
1,38,20,MSH,C01,Infections
2,8033,20,MSH,C01,Infections
3,13566,20,MSH,C01,Infections
4,39,20,MSH,C01,Infections
...,...,...,...,...,...
38319,20669,37,MSH,F03,Mental Disorders
38320,40,37,MSH,F03,Mental Disorders
38321,5209,37,MSH,F03,Mental Disorders
38322,4771,37,MSH,F03,Mental Disorders


## geneAttributes table

In [18]:
dict_all_tables['geneAttributes']

,geneNID,geneId,geneName,geneDescription,pLI,DSI,DPI
0,1,1,A1BG,alpha-1-B glycoprotein,4.991700e-09,0.700,0.538
1,2,2,A2M,alpha-2-macroglobulin,4.522900e-11,0.529,0.769
2,3,3,A2MP1,alpha-2-macroglobulin pseudogene 1,NaN,NaN,NaN
3,4,9,NAT1,N-acetyltransferase 1,1.929400e-14,0.536,0.846
4,5,10,NAT2,N-acetyltransferase 2,3.274400e-06,0.451,0.885
...,...,...,...,...,...,...,...
26132,26133,115482722,H3P33,H3 histone pseudogene 33,NaN,0.736,0.346
26133,26134,115482723,H3P40,H3 histone pseudogene 40,NaN,0.506,0.846
26134,26135,115561815,LNC-LBCS,"lncRNA bladder and prostate cancer suppressor,...",NaN,0.890,0.077
26135,26136,115804232,CEROX1,cytoplasmic endogenous regulator of oxidative ...,NaN,NaN,NaN


## geneDiseaseNetwork table

In [ ]:
dict_all_tables['geneDiseaseNetwork']

In [20]:
dict_all_tables['geneDiseaseNetwork']

,NID,diseaseNID,geneNID,source,association,associationType,sentence,pmid,score,EL,EI,year
0,1130681,2107,793,BEFREE,NaN,Biomarker,No correlation could be found between Broder's...,1000501.0,0.10,None,0.956175,1976
1,261998,431,775,BEFREE,NaN,GeneticVariation,"However, there are few reports describing soma...",10021299.0,0.40,None,0.987013,1999
2,369637,450,775,BEFREE,NaN,GeneticVariation,WAF1 genotype and endometrial cancer susceptib...,10021299.0,0.02,None,1.000000,1999
3,2128526,10033,775,BEFREE,NaN,GeneticVariation,WAF1 genotype and endometrial cancer susceptib...,10021299.0,0.07,None,1.000000,1999
4,262799,431,7385,BEFREE,NaN,Biomarker,APC and APC2 may therefore have comparable fun...,10021369.0,0.01,None,1.000000,1999
...,...,...,...,...,...,...,...,...,...,...,...,...
3261319,3261082,30235,1698,ORPHANET,NaN,ChromosomalRearrangement,None,NaN,0.30,None,NaN,NA
3261320,3261111,30244,4952,CLINVAR,NaN,GeneticVariation,None,NaN,0.20,None,1.000000,NA
3261321,3261162,30255,5756,GENOMICS_ENGLAND,NaN,Biomarker,None,NaN,0.30,strong,NaN,NA
3261322,3261163,30255,5756,GENOMICS_ENGLAND,NaN,Biomarker,None,NaN,0.30,strong,NaN,NA


In [27]:
dict_all_tables['geneDiseaseNetwork']['source'].unique()

array(['BEFREE', 'MGD', 'UNIPROT', 'CTD_human', 'RGD', 'CLINVAR',
       'CLINGEN', 'GENOMICS_ENGLAND', 'ORPHANET', 'PSYGENET', 'LHGDN',
       'CTD_mouse', 'GWASDB', 'GWASCAT', 'CTD_rat', 'HPO', 'CGI'],
      dtype=object)

#### Testing an idea:

In [42]:
df_emily_geneDiseaseNetwork_to_geneAttributes = (
    pd.merge(
        dict_all_tables['geneDiseaseNetwork'],
        dict_all_tables['geneAttributes'],
        on = ['geneNID'],
        how = 'left',
    )
    .rename(
        columns = {
            'pLI' : 'gene_pLI',
            'DSI' : 'gene_DSI',
            'DPI' : 'gene_DPI',
        }
    )
)

In [43]:
df_emily_geneDiseaseNetwork_to_geneAttributes

,NID,diseaseNID,geneNID,source,association,associationType,sentence,pmid,score,EL,EI,year,geneId,geneName,geneDescription,gene_pLI,gene_DSI,gene_DPI
0,1130681,2107,793,BEFREE,NaN,Biomarker,No correlation could be found between Broder's...,1000501.0,0.10,None,0.956175,1976,1048,CEACAM5,CEA cell adhesion molecule 5,4.162500e-10,0.420,0.846
1,261998,431,775,BEFREE,NaN,GeneticVariation,"However, there are few reports describing soma...",10021299.0,0.40,None,0.987013,1999,1026,CDKN1A,cyclin dependent kinase inhibitor 1A,1.857500e-03,0.403,0.769
2,369637,450,775,BEFREE,NaN,GeneticVariation,WAF1 genotype and endometrial cancer susceptib...,10021299.0,0.02,None,1.000000,1999,1026,CDKN1A,cyclin dependent kinase inhibitor 1A,1.857500e-03,0.403,0.769
3,2128526,10033,775,BEFREE,NaN,GeneticVariation,WAF1 genotype and endometrial cancer susceptib...,10021299.0,0.07,None,1.000000,1999,1026,CDKN1A,cyclin dependent kinase inhibitor 1A,1.857500e-03,0.403,0.769
4,262799,431,7385,BEFREE,NaN,Biomarker,APC and APC2 may therefore have comparable fun...,10021369.0,0.01,None,1.000000,1999,10297,APC2,APC regulator of WNT signaling pathway 2,9.999400e-01,0.582,0.654
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3261319,3261082,30235,1698,ORPHANET,NaN,ChromosomalRearrangement,None,NaN,0.30,None,NaN,NA,2290,FOXG1,forkhead box G1,9.435500e-01,0.522,0.692
3261320,3261111,30244,4952,CLINVAR,NaN,GeneticVariation,None,NaN,0.20,None,1.000000,NA,6792,CDKL5,cyclin dependent kinase like 5,9.993200e-01,0.522,0.692
3261321,3261162,30255,5756,GENOMICS_ENGLAND,NaN,Biomarker,None,NaN,0.30,strong,NaN,NA,8092,ALX1,ALX homeobox 1,1.761800e-01,0.636,0.462
3261322,3261163,30255,5756,GENOMICS_ENGLAND,NaN,Biomarker,None,NaN,0.30,strong,NaN,NA,8092,ALX1,ALX homeobox 1,1.761800e-01,0.636,0.462


In [38]:
df_emily_geneDiseaseNetwork_to_geneAttributes.columns

Index(['NID', 'diseaseNID', 'geneNID', 'source', 'association',
       'associationType', 'sentence', 'pmid', 'score', 'EL', 'EI', 'year',
       'geneId', 'geneName', 'geneDescription', 'gene_pLI', 'gene_DSI',
       'gene_DPI'],
      dtype='object')

## variantAttributes table

In [22]:
dict_all_tables['variantAttributes']

,variantNID,variantId,s,chromosome,coord,most_severe_consequence,DSI,DPI
0,12361,rs1000005,snv,21,33060745,intron_variant,\N,\N
1,64042,rs10000770,snv,4,142693109,intron_variant,1.0,0.04
2,166562,rs1000091588,snv,11,68935374,stop_gained,1.0,0.12
3,12362,rs1000096,snv,4,38691214,intron_variant,\N,\N
4,64043,rs10001106,snv,4,10125817,intergenic_variant,0.925,0.12
...,...,...,...,...,...,...,...,...
194510,12359,rs999943,snv,6,33656956,intron_variant,1.0,0.08
194511,12360,rs999944,snv,2,64822719,intergenic_variant,1.0,0.08
194512,166561,rs999947969,snv,7,80671145,synonymous_variant,0.827,0.08
194513,64040,rs9999653,snv,4,55632082,intron_variant,1.0,0.08


## variantGene table

In [23]:
dict_all_tables['variantGene']

,geneNID,variantNID
0,6229,64042
1,2550,166562
2,10146,12362
3,17615,12363
4,8552,64044
...,...,...
172422,16129,64039
172423,10675,166560
172424,2698,12359
172425,705,166561


## variantDiseaseNetwork table

In [24]:
dict_all_tables['variantDiseaseNetwork']

,NID,diseaseNID,variantNID,source,association,associationType,sentence,pmid,score,EI,year
0,1,8252,105630,UNIPROT,0.0,GeneticVariation,"When expressed in heterologous cells, both Gln...",10022417.0,0.82,1.0,1999
1,2,8252,105631,UNIPROT,0.0,GeneticVariation,"When expressed in heterologous cells, both Gln...",10022417.0,0.82,1.0,1999
2,3,8252,105632,UNIPROT,NaN,GeneticVariation,The same molecular defects of the gonadotropin...,10022417.0,0.80,1.0,1999
3,4,8252,105633,UNIPROT,0.0,GeneticVariation,"When expressed in heterologous cells, both Gln...",10022417.0,0.80,1.0,1999
4,5,8252,105634,UNIPROT,NaN,GeneticVariation,The same molecular defects of the gonadotropin...,10022417.0,0.80,1.0,1999
...,...,...,...,...,...,...,...,...,...,...,...
751427,751428,14662,166551,CLINVAR,NaN,CausalMutation,None,NaN,0.70,NaN,NA
751428,751429,8794,166553,CLINVAR,NaN,CausalMutation,None,NaN,0.70,NaN,NA
751429,751430,14928,166554,CLINVAR,NaN,GeneticVariation,None,NaN,0.70,NaN,NA
751430,751431,397,166557,UNIPROT,NaN,GeneticVariation,None,NaN,0.70,NaN,NA


In [25]:
dict_all_tables['variantDiseaseNetwork']['source'].unique()

array(['UNIPROT', 'CLINVAR', 'BEFREE', 'GWASDB', 'GWASCAT'], dtype=object)